# BLSTM Notebook

In [53]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional, Embedding, Input, TimeDistributed
from source.model.load_data import train_test_split, train_test_split_LSTM

from source.model.scoring_metrics import get_windiff, get_pk, get_k_kappa

In [56]:
# these are hyperparameters. 
# TODO: We should try to find good values for thos
batch_size = 64
n_timesteps = 20
train_ratio = 0.6 # ratio of meetings the model is trained on
LSTM_units = 20

datasets = """Bed002 Bed003 Bed004 Bed005 Bed006 Bed008 Bed009 Bed010 Bed011 Bed012 Bed013 Bed014 Bed015 Bed016 Bed017 Bmr001 Bmr002 Bmr005 Bmr007 Bmr009 Bmr010 Bmr011 Bmr012 Bmr013 Bmr014 Bmr018 Bmr019 Bmr021 Bmr022 Bmr024 Bmr025 Bmr026 Bmr027 Bmr029 Bns001 Bns002""".split(" ")
results_merged_path = "./results_merged_fixedf0/"

In [57]:
#load the data
X_train_df, Y_train_df, X_test_df, Y_test_df= train_test_split_LSTM(datasets, results_merged_path, n_timesteps, split=train_ratio)

print(X_train_df.shape)
print(Y_train_df.shape)
print(X_test_df.shape)
print(Y_test_df.shape)

# convert data from dataframes into numpy arrays of in a 3-D shape (samples, timesteps, features)
X_train = X_train_df.values.astype('float32').reshape(-1, n_timesteps, 5)
X_test = X_test_df.values.astype('float32').reshape(-1, n_timesteps, 5)
Y_train = Y_train_df.values.astype('float32').reshape(-1, n_timesteps, 1)
Y_test = Y_test_df.values.astype('float32').reshape(-1, n_timesteps, 1)

(20000, 5)
(20000, 1)
(30500, 5)
(30500, 1)


In [ ]:
# build the model
model = Sequential()
model.add(Bidirectional(LSTM(LSTM_units, return_sequences=True), input_shape=(n_timesteps, 5)))
model.add(TimeDistributed(Dense(1, activation='sigmoid')))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#train the model
history=model.fit(X_train, Y_train,
           batch_size=batch_size,
           epochs=100,
        #    class_weight= {0:1, 1:10},
        #    sample_weight_mode='temporal',
           validation_data=[X_test, Y_test])

TypeError: fit() got an unexpected keyword argument 'sample_weight_mode'

In [ ]:
def find_threshold(predictions, Y_test):
    thresholds_to_try = [0.0001, 0.0003, 0.001, 0.003, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.2, 0.3, 0.4, 0.5]
    windiffs = []
    min_windiff = 1.0
    best_threshold = 0
    for threshold in thresholds_to_try:
        pred = (predictions >= threshold).astype(float)
        windiff = get_windiff(Y_test, pred)
        windiffs.append(windiff)
        if windiff < min_windiff:
            min_windiff = windiff
            best_threshold = threshold
    print("windiffs with differnt thresholds:", windiffs)
    print("min_windiff:", min_windiff)
    print("best_threshold:", best_threshold)
    return best_threshold

In [ ]:
#measure the performance
predictions = model(X_test).numpy()

predictions = predictions.flatten()

pred_eval = (predictions >= 0.5).astype(float)
Y_test = Y_test.flatten()

In [ ]:
threshold = find_threshold(predictions, Y_test)
pred_eval = (predictions >= threshold).astype(float)

[1.0, 1.0, 1.0, 1.0, 0.9230486623693264, 0.8294980127584249, 0.8041481580441535, 0.7884840185698541, 0.7449316990080491, 0.600681339968605, 0.47109314986139406, 0.4285094018235864, 0.40589826659096223, 0.4082695968738519, 0.40245816772986875, 0.4010220099529074, 0.4010220099529074, 0.4010220099529074]
min_windiff: 0.4010220099529074
best_threshold: 0.3


In [ ]:
print("predictions:", predictions)
print("pred_eval:", pred_eval)
print("# boundaries predicted:", np.count_nonzero(pred_eval))

[0.04482847 0.03519651 0.0282278  ... 0.05078569 0.06053896 0.06993654]
[0. 0. 0. ... 0. 0. 0.]


In [ ]:
windiff = get_windiff(Y_test, predictions)
pk = get_pk(Y_test, predictions, k=58)
k_k = get_k_kappa(Y_test, predictions)
print("windiff:", windiff)
print("pk:", pk)
print("K_k ", k_k)
print("windiff_classifier:", get_windiff(Y_test, pred_eval))
print("pk_classifier:", get_pk(Y_test, pred_eval))
print("K_k_classifier ", get_k_kappa(Y_test, pred_eval))

windiff: 0.4010220099529074
pk: 0.3913769495374545
K_k  0.3304895728783317
windiff_classifier: 0.4010220099529074
pk_classifier: 0.4010220099529074
K_k_classifier  0.0
